In [233]:
from tqdm import tqdm
tqdm.pandas(desc="Progress")

In [265]:
# def get_coeffs_from_corr(corr_abs_list):
#     inv_abs_list = [1/a for a in corr_abs_list]
#     l = [a/sum(inv_abs_list) for a in inv_abs_list]
#     return l

# # get_coeffs_from_corr([5,15.3,5, 20.6,20.2]) # capital-oc
# get_coeffs_from_corr([27.05,29.81,36.63]) # raif

In [266]:
# import geopy.distance

# coords_1 = (58.010258, 56.234203)
# coords_2 = (57.9438700, 56.05454969)

# geopy.distance.geodesic(coords_1, coords_2).km

12.948899946902948

In [273]:
from math import *

def distance(x,y):
    """
    Параметры
    ----------
    x : tuple, широта и долгота первой геокоординаты 
    y : tuple, широта и долгота второй геокоординаты 
    
    Результат
    ----------
    result : дистанция в километрах между двумя геокоординатами
    """
    R = 6373.0 # радиус земли в километрах

    lat_a, long_a, lat_b, long_b = map(radians, [*x,*y])    
    
    dlon = long_b - long_a
    dlat = lat_b - lat_a
    a = sin(dlat/2)**2 + cos(lat_a) * cos(lat_b) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a)) # c = 2 * asin(sqrt(a)) # the same 
    return R * c

distance((58.010258, 56.234203), (57.94387008249369, 56.0545496986542) )

12.915277388593745

In [268]:
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance in kilometers between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
    return c * r

haversine(56.234203, 58.010258, 56.0545496986542, 57.94387008249369)

12.911224265295896

In [269]:
def dist(lat1, long1, lat2, long2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lat1, long1, lat2, long2 = map(radians, [lat1, long1, lat2, long2])
    # haversine formula 
    dlon = long2 - long1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    # Radius of earth in kilometers is 6371
    km = 6371* c
    return km

distance((58.010258, 56.234203), (57.94387008249369, 56.0545496986542) )

12.915277388593745

In [271]:
import pyproj
geodesic = pyproj.Geod(ellps='WGS84')
fwd_azimuth, _, distance = geodesic.inv(56.234203, 58.010258,  56.0545496986542, 57.94387008249369) # lon, lat !!!!!!
distance = distance/1000
fwd_azimuth, distance

(-124.74531992770093, 12.948894270028136)

In [272]:
geodesic = pyproj.Geod(ellps='WGS84')

def dist(lat1, long1, lat2, long2):
    fwd_azimuth, _, distance = geodesic.inv(lat1, long1, lat2, long2)
    distance = distance/1000
    return distance

In [178]:
import pandas as pd
import numpy as np
geo = pd.DataFrame({"lat":[3.12341, 4.3452345], 'lon': [2.12341, 2.3442345], 'km': [0,1]})

In [181]:
def find_nearest(df, lat, long, feat, n_near=5):
    
    distances = df.apply(
        lambda row: dist(lat, long, row['lat'], row['lon']), 
        axis=1)
    
#     assert list(distances).count(0)==1
    
    ind_sort  = np.argsort(distances)
    
    sort_dist = distances[ind_sort]
    sort_dist = np.array(sort_dist)
    
    filt_cnt = len(sort_dist[np.where( sort_dist < 139)])

    ## by feature
#     top_ind_sort = ind_sort[0:1+n_near]
#     return geo.loc[top_ind_sort, feat].tolist()

    # distances
#     top_sort_dist = sort_dist[0:1+n_near]
    top_sort_dist = sort_dist[0:filt_cnt]
    return list(top_sort_dist)

    ## coordinates
#     top_ind_sort = ind_sort[0:1+n_near]
#     return list(zip(geo.loc[top_ind_sort, 'lat'].tolist(),geo.loc[top_ind_sort, 'lon'].tolist()))

In [183]:
# geo['dist'] = geo.progress_apply(
#     lambda row: find_nearest(row['lat'], row['lon'], feat='lat', n_near=5),
#     axis=1)

# geo

In [174]:
bar_df = pd.read_csv('overpass_out/omsnx_city_dict_bar_ALL_DF.csv')

In [182]:
bar_df['dist'] = bar_df.progress_apply(
    lambda row: find_nearest(bar_df, row['lat'], row['lon'], feat='lat', n_near=5),
    axis=1)

Progress: 100%|██████████| 4007/4007 [05:39<00:00, 11.79it/s]


In [279]:
import pandas as pd
import numpy as np

from sklearn.neighbors import BallTree
from io import StringIO

# Create DataFrame from you lat/lon dataset
data = """NAME Latitude Longitude
B 58.010258 56.234203
"""

# Use StringIO to allow reading of string as CSV
# df = pd.read_csv(StringIO(data), sep = ' ')
df = bar_df.copy()

# Setup Balltree using df as reference dataset
# Use Haversine calculate distance between points on the earth from lat/long
# haversine - https://pypi.org/project/haversine/ 
tree = BallTree(np.deg2rad(df[['lat', 'lon']].values), metric='haversine')

# Setup distance queries (points for which we want to find nearest neighbors)
# other_data = """NAME Latitude Longitude
# C_alt C 57.94387008249369 56.0545496986542"""

# df_other = pd.read_csv(StringIO(other_data), sep = ' ')
df_other = bar_df.copy()

query_lats = df_other['lat']
query_lons = df_other['lon']

# Find closest city in reference dataset for each in df_other
# use k = 3 for 3 closest neighbors
distances, indices = tree.query(np.deg2rad(np.c_[query_lats, query_lons]), k = 3)

r_km = 6371 # multiplier to convert to km (from unit distance)
for name, d, ind in zip(df_other.index, distances, indices):
  print(f"NAME {name} closest matches:")
  for i, index in enumerate(ind):
    print(f"\t{df.index[index]} with distance {d[i]*r_km:.4f} km")

NAME 0 closest matches:
	0 with distance 0.0000 km
	3335 with distance 0.0054 km
	3275 with distance 0.0666 km
NAME 1 closest matches:
	1 with distance 0.0000 km
	3282 with distance 0.1425 km
	3297 with distance 0.1708 km
NAME 2 closest matches:
	2 with distance 0.0000 km
	3290 with distance 0.0431 km
	3298 with distance 0.1111 km
NAME 3 closest matches:
	3 with distance 0.0000 km
	4 with distance 0.0411 km
	16 with distance 0.1285 km
NAME 4 closest matches:
	4 with distance 0.0000 km
	3 with distance 0.0411 km
	30 with distance 0.1169 km
NAME 5 closest matches:
	5 with distance 0.0000 km
	7 with distance 0.0116 km
	22 with distance 0.1017 km
NAME 6 closest matches:
	6 with distance 0.0000 km
	11 with distance 0.1159 km
	24 with distance 0.1226 km
NAME 7 closest matches:
	7 with distance 0.0000 km
	5 with distance 0.0116 km
	22 with distance 0.0901 km
NAME 8 closest matches:
	8 with distance 0.0000 km
	31 with distance 0.0063 km
	25 with distance 0.2083 km
NAME 9 closest matches:
	9 wi

	628 with distance 0.0000 km
	624 with distance 0.0179 km
	3835 with distance 0.1443 km
NAME 629 closest matches:
	629 with distance 0.0000 km
	622 with distance 0.3540 km
	2031 with distance 0.6558 km
NAME 630 closest matches:
	630 with distance 0.0000 km
	640 with distance 0.0576 km
	649 with distance 0.4653 km
NAME 631 closest matches:
	631 with distance 0.0000 km
	635 with distance 0.5634 km
	632 with distance 1.4363 km
NAME 632 closest matches:
	632 with distance 0.0000 km
	646 with distance 0.1678 km
	647 with distance 0.3557 km
NAME 633 closest matches:
	633 with distance 0.0000 km
	634 with distance 0.0364 km
	642 with distance 0.5134 km
NAME 634 closest matches:
	634 with distance 0.0000 km
	633 with distance 0.0364 km
	642 with distance 0.4771 km
NAME 635 closest matches:
	635 with distance 0.0000 km
	631 with distance 0.5634 km
	632 with distance 0.8730 km
NAME 636 closest matches:
	636 with distance 0.0000 km
	643 with distance 0.0114 km
	2114 with distance 0.8002 km
NAME 6

NAME 1288 closest matches:
	1288 with distance 0.0000 km
	1236 with distance 0.4249 km
	1373 with distance 0.8851 km
NAME 1289 closest matches:
	1289 with distance 0.0000 km
	1471 with distance 0.0015 km
	1477 with distance 0.0076 km
NAME 1290 closest matches:
	1290 with distance 0.0000 km
	1208 with distance 0.1397 km
	1139 with distance 0.6821 km
NAME 1291 closest matches:
	1291 with distance 0.0000 km
	1511 with distance 0.0024 km
	1154 with distance 0.0044 km
NAME 1292 closest matches:
	1292 with distance 0.0000 km
	1145 with distance 0.0201 km
	1664 with distance 0.1272 km
NAME 1293 closest matches:
	1293 with distance 0.0000 km
	1755 with distance 0.0390 km
	1576 with distance 0.0995 km
NAME 1294 closest matches:
	1294 with distance 0.0000 km
	1261 with distance 0.1201 km
	1084 with distance 0.1559 km
NAME 1295 closest matches:
	1295 with distance 0.0000 km
	1356 with distance 0.0314 km
	1217 with distance 0.0368 km
NAME 1296 closest matches:
	1296 with distance 0.0000 km
	1278 w

	1892 with distance 0.0000 km
	1933 with distance 0.0394 km
	1874 with distance 0.0748 km
NAME 1893 closest matches:
	1893 with distance 0.0000 km
	1921 with distance 0.0811 km
	1912 with distance 0.1165 km
NAME 1894 closest matches:
	1894 with distance 0.0000 km
	1883 with distance 0.1252 km
	1884 with distance 0.3009 km
NAME 1895 closest matches:
	1895 with distance 0.0000 km
	1924 with distance 0.0150 km
	1922 with distance 0.0877 km
NAME 1896 closest matches:
	1896 with distance 0.0000 km
	1940 with distance 0.0323 km
	941 with distance 0.1577 km
NAME 1897 closest matches:
	1897 with distance 0.0000 km
	1929 with distance 0.0580 km
	1937 with distance 0.0655 km
NAME 1898 closest matches:
	1898 with distance 0.0000 km
	1907 with distance 0.0105 km
	1946 with distance 0.0464 km
NAME 1899 closest matches:
	1899 with distance 0.0000 km
	1946 with distance 0.0336 km
	1898 with distance 0.0800 km
NAME 1900 closest matches:
	1900 with distance 0.0000 km
	1935 with distance 0.0104 km
	1932

NAME 2526 closest matches:
	2526 with distance 0.0000 km
	2937 with distance 0.0001 km
	2674 with distance 0.0007 km
NAME 2527 closest matches:
	2527 with distance 0.0000 km
	2443 with distance 0.0018 km
	2486 with distance 0.0019 km
NAME 2528 closest matches:
	2528 with distance 0.0000 km
	3023 with distance 0.0003 km
	2728 with distance 0.0008 km
NAME 2529 closest matches:
	2529 with distance 0.0000 km
	2574 with distance 0.0261 km
	2901 with distance 0.0275 km
NAME 2530 closest matches:
	2530 with distance 0.0000 km
	2531 with distance 0.1638 km
	2568 with distance 0.1828 km
NAME 2531 closest matches:
	2531 with distance 0.0000 km
	2568 with distance 0.0190 km
	2879 with distance 0.0404 km
NAME 2532 closest matches:
	2532 with distance 0.0000 km
	3047 with distance 0.0363 km
	2493 with distance 0.0900 km
NAME 2533 closest matches:
	2533 with distance 0.0000 km
	2454 with distance 0.0078 km
	2907 with distance 0.0434 km
NAME 2534 closest matches:
	2534 with distance 0.0000 km
	2850 w

	3146 with distance 0.0412 km
	3198 with distance 0.0560 km
NAME 3160 closest matches:
	3160 with distance 0.0000 km
	3219 with distance 0.0062 km
	3227 with distance 0.0104 km
NAME 3161 closest matches:
	3161 with distance 0.0000 km
	3189 with distance 0.0018 km
	3617 with distance 0.0029 km
NAME 3162 closest matches:
	3162 with distance 0.0000 km
	3244 with distance 0.0539 km
	3137 with distance 0.0810 km
NAME 3163 closest matches:
	3163 with distance 0.0000 km
	3185 with distance 0.0046 km
	3739 with distance 0.1325 km
NAME 3164 closest matches:
	3164 with distance 0.0000 km
	3205 with distance 0.1672 km
	3139 with distance 0.2294 km
NAME 3165 closest matches:
	3165 with distance 0.0000 km
	3197 with distance 0.1776 km
	3184 with distance 0.6653 km
NAME 3166 closest matches:
	3166 with distance 0.0000 km
	3142 with distance 0.0537 km
	3222 with distance 0.0730 km
NAME 3167 closest matches:
	3167 with distance 0.0000 km
	3738 with distance 0.0098 km
	3723 with distance 0.0229 km
NAME

	3827 with distance 0.0000 km
	3826 with distance 0.0209 km
	3828 with distance 0.0344 km
NAME 3828 closest matches:
	3828 with distance 0.0000 km
	3825 with distance 0.0011 km
	3832 with distance 0.0164 km
NAME 3829 closest matches:
	3829 with distance 0.0000 km
	3833 with distance 2.3760 km
	3831 with distance 2.4050 km
NAME 3830 closest matches:
	3830 with distance 0.0000 km
	3826 with distance 2.4705 km
	3827 with distance 2.4914 km
NAME 3831 closest matches:
	3831 with distance 0.0000 km
	3833 with distance 0.0291 km
	3832 with distance 0.1072 km
NAME 3832 closest matches:
	3832 with distance 0.0000 km
	3825 with distance 0.0154 km
	3828 with distance 0.0164 km
NAME 3833 closest matches:
	3833 with distance 0.0000 km
	3831 with distance 0.0291 km
	3832 with distance 0.1362 km
NAME 3834 closest matches:
	3834 with distance 0.0000 km
	3839 with distance 0.4872 km
	3835 with distance 0.5861 km
NAME 3835 closest matches:
	3835 with distance 0.0000 km
	3839 with distance 0.0989 km
	628